<a href="https://colab.research.google.com/github/spencer18001/Clustering-And-Dimensionality-Reduction---Deep-Dive/blob/main/10/10_note_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Density Based Clustering Part2

160. Density based clustering validation (DBCV) - part 1
161. Density based clustering validation (DBCV) - part 2
162. Demo: Implementing DBCV from scratch - part 1
163. Demo: Implementing DBCV from scratch - part 2 + DBCV python function
164. DBSCAN general guideliness
165. Demo: Clustering digits (mnist784) with DBSCAN
166. Demo: Clustering animals with DBSCAN (categorical data)
167. HDBSCAN clustering algorithm - part 1
168. HDBSCAN clustering algorithm - part 2
169. HDBSCAN clustering algorithm - part 3

#### Density based clustering validation (DBCV)

core distance

mutual reachability distance

mutual reachability graph

mutual reachability graph minimum spanning tree

cluster density sparseness

cluster density separation

cluster validation coefficient